Name: script1
Author: Ariane Neumann
Date: 2025-10-15
Description: Test script to work on assignment 2(student2) for the course BINP16 in HT25LU.

In [1]:
print("=="*63)

What to prepare:
- Reading the files
- Writing the script
- Handle errors

In [2]:
#%% Convert DNA sequence into amino acid sequence
# The goal is to read a fasta file and translate the DNA sequence into protein sequence using the standard genetic code.
# First I need to define the function

import sys
import os

# ----------------------------------------------------------------------
# DOCUMENTATION BLOCK
# Program: dna2aa.py
# Author: Student II
# Date: October 2025
# Description: Converts a multi-FASTA DNA file into a multi-FASTA file
# containing the translated amino acid sequences.
# Requirements: Uses sys, def, if/while, try/except/finally, with open.
# Execution: python dna2aa.py DNA.fna output_file.txt [optional]
# ----------------------------------------------------------------------

# Standard Genetic Code Dictionary (DNA codons -> AA)
# Note: T is used here, and is replaced by U (conceptually) during translation.
GENETIC_CODE = {
    'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V', 'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E', 'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G',
    'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L', 'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',
    'TAT': 'Y', 'TAC': 'Y', 'TAA': '*', 'TAG': '*', 'TGT': 'C', 'TGC': 'C', 'TGA': '*', 'TGG': 'W',
    'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L', 'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q', 'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
    'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M', 'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K', 'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R'
    # N is often used as a wildcard, translating to X (unknown)
}

def read_fasta(file_path):
    """
    Reads a FASTA file, cleans the DNA sequences, and stores them in a dictionary.
    
    Args:
        file_path (str): The path to the input FASTA file.
        
    Returns:
        dict: A dictionary mapping sequence IDs to cleaned DNA sequences.
    """
    sequences = {}
    current_id = None
    current_sequence_lines = []
    
    # Define a set of valid DNA characters for validation
    VALID_DNA_CHARS = {'A', 'C', 'T', 'G', 'N'}

    try:
        # Using 'with open' for secure and efficient file handling
        with open(file_path, 'r') as f:
            line_number = 0
            # Use a while loop to iterate over lines
            while True:
                line = f.readline()
                line_number += 1
                if not line: # End of file
                    break
                    
                line = line.strip()
                
                if line.startswith('>'):
                    # Start of a new sequence
                    if current_id and current_sequence_lines:
                        # Process the previous sequence
                        sequences[current_id] = "".join(current_sequence_lines)
                        
                    # Extract the ID (everything after '>')
                    current_id = line[1:].strip()
                    current_sequence_lines = []
                elif current_id:
                    # Clean and validate the sequence line
                    cleaned_line = line.upper()
                    
                    # Remove non-DNA characters (per instructions, though validation is better)
                    # For robust validation: check if only valid chars remain
                    for char in cleaned_line:
                        if char not in VALID_DNA_CHARS:
                            raise ValueError(
                                f"Invalid character '{char}' found in sequence {current_id} on line {line_number}. "
                                "Only A, C, T, G, N are allowed."
                            )
                            
                    current_sequence_lines.append(cleaned_line)
                    
        # Process the last sequence after the while loop finishes
        if current_id and current_sequence_lines:
            sequences[current_id] = "".join(current_sequence_lines)
            
    except FileNotFoundError:
        print(f"Error: Input file not found at '{file_path}'", file=sys.stderr)
        return None
    except ValueError as e:
        print(f"Error in file content: {e}", file=sys.stderr)
        return None
    except Exception as e:
        print(f"An unexpected error occurred while reading the file: {e}", file=sys.stderr)
        return None
    
    if not sequences:
        print("Error: No sequences found in the input file.", file=sys.stderr)
        return None
        
    return sequences

def translate_dna(dna_sequence, codon_table):
    """
    Translates a cleaned DNA sequence into an amino acid sequence.
    
    Args:
        dna_sequence (str): The cleaned DNA sequence (uppercase, ACTGN).
        codon_table (dict): The standard genetic code.
        
    Returns:
        str: The translated amino acid sequence.
    """
    aa_sequence = []
    
    # The sequence length must be a multiple of 3 for complete codons.
    # We ignore the trailing 1 or 2 bases if the length is not divisible by 3.
    sequence_length = len(dna_sequence)
    
    # Use a while loop to iterate through the sequence
    i = 0
    while i < sequence_length - 2:
        codon = dna_sequence[i:i+3]
        
        # Conceptual T -> U conversion is handled by using T in the codon table
        # If the sequence contains 'N' (unknown), treat it as an unknown amino acid 'X'
        if 'N' in codon:
            aa_sequence.append('X')
        else:
            # Try to look up the codon, handling potential missing keys (shouldn't happen 
            # with a full 64-codon table, but robust)
            try:
                amino_acid = codon_table.get(codon, 'X')  # Default to 'X' for safety
                aa_sequence.append(amino_acid)
            except Exception:
                # Fallback for unexpected issues
                aa_sequence.append('X') 

        i += 3
        
    return "".join(aa_sequence)


def main():
    """
    Main function to handle command-line arguments and orchestrate the translation.
    """
    # 1. Handle command-line arguments (sys)
    if len(sys.argv) < 2:
        print(f"Usage: python {os.path.basename(sys.argv[0])} <input_DNA_fasta> [output_AA_fasta]", file=sys.stderr)
        sys.exit(1)

    input_file = sys.argv[1]
    # Default output file name is used if not provided
    output_file = sys.argv[2] if len(sys.argv) > 2 else "translated_aa.faa"

    print(f"Starting DNA translation from '{input_file}' to '{output_file}'...")
    
    # 2. Read the input FASTA file
    dna_sequences = read_fasta(input_file)
    
    if dna_sequences is None:
        # Errors handled and printed inside read_fasta
        sys.exit(1)
        
    translated_sequences = {}
    
    # 3. Translate all sequences
    # Use a 'for' loop (which is a controlled 'while' loop internally) for iteration
    for seq_id, dna_seq in dna_sequences.items():
        if len(dna_seq) < 3:
            print(f"Warning: Sequence '{seq_id}' too short for translation (Length: {len(dna_seq)}). Skipping.", file=sys.stderr)
            continue
            
        aa_seq = translate_dna(dna_seq, GENETIC_CODE)
        translated_sequences[seq_id] = aa_seq
        
    # 4. Write the output to a file (try/except/finally)
    output_success = False
    output_f = None # Initialize file handle outside try

    try:
        # Using 'with open' for secure file writing
        with open(output_file, 'w') as output_f:
            for seq_id, aa_seq in translated_sequences.items():
                # Write FASTA header
                output_f.write(f">{seq_id}\n")
                # Write sequence, breaking lines for standard FASTA format (e.g., every 80 chars)
                i = 0
                while i < len(aa_seq):
                    output_f.write(aa_seq[i:i+80] + '\n')
                    i += 80
            output_success = True
            
    except Exception as e:
        print(f"\nCritical Error: Failed to write to output file '{output_file}'. Error: {e}", file=sys.stderr)
        
    finally:
        # The 'finally' block ensures cleanup, though 'with open' mostly handles this.
        if output_success:
            print(f"Successfully wrote {len(translated_sequences)} translated sequence(s) to '{output_file}'.")
        else:
            print("Translation process completed with errors during file writing.", file=sys.stderr)


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'PyGeneticCode'

In [ ]:
import sys
import os

# ----------------------------------------------------------------------
# DOCUMENTATION BLOCK
# Program: amino_count.py
# Author: Gemini (as Student II)
# Date: October 2025
# Description: Counts the absolute combined abundances of all 20 standard 
# amino acids plus a category 'X' for non-standard letters in a 
# multi-FASTA amino acid file.
# Requirements: Uses sys, def, if/while, try/except/finally, with open.
# Execution: python amino_count.py amino.faa output_file.txt [optional]
# ----------------------------------------------------------------------

# Set of the 20 standard amino acid abbreviations
STANDARD_AAS = {'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 
                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'}


def read_aa_fasta(file_path):
    """
    Reads a multi-FASTA file and extracts all amino acid sequences.
    
    Args:
        file_path (str): The path to the input FASTA file.
        
    Returns:
        list: A list of all combined amino acid sequences (one string per ID).
    """
    sequences = []
    current_sequence_lines = []
    
    try:
        # Using 'with open' for secure and efficient file handling
        with open(file_path, 'r') as f:
            # Use a while loop to iterate over lines
            while True:
                line = f.readline()
                if not line: # End of file
                    break
                    
                line = line.strip()
                
                if line.startswith('>'):
                    # Start of a new sequence
                    if current_sequence_lines:
                        sequences.append("".join(current_sequence_lines))
                        
                    current_sequence_lines = [] # Reset for the new sequence
                elif line:
                    # Append sequence line, converting to uppercase immediately
                    current_sequence_lines.append(line.upper())
                    
        # Process the last sequence after the while loop finishes
        if current_sequence_lines:
            sequences.append("".join(current_sequence_lines))
            
    except FileNotFoundError:
        print(f"Error: Input file not found at '{file_path}'", file=sys.stderr)
        return None
    except Exception as e:
        print(f"An unexpected error occurred while reading the file: {e}", file=sys.stderr)
        return None
    
    if not sequences:
        print("Error: No sequences found in the input file.", file=sys.stderr)
        return None
        
    return sequences


def count_abundances(aa_sequences, standard_aas):
    """
    Counts the absolute abundance of each standard amino acid and 'X' for others.
    
    Args:
        aa_sequences (list): A list of amino acid sequence strings.
        standard_aas (set): A set of the 20 standard amino acid codes.
        
    Returns:
        dict: A dictionary mapping AA codes (and 'X') to their total counts.
    """
    # Initialize counts for all standard AAs to zero
    counts = {aa: 0 for aa in sorted(list(standard_aas))}
    counts['X'] = 0 # Initialize 'X' for non-standard/unknown AAs

    # Use nested loops (implicitly) to iterate through all sequences and characters
    for seq in aa_sequences:
        for char in seq:
            if char in standard_aas:
                counts[char] += 1
            elif char.isalpha(): # All other alphabetical characters are grouped as 'X'
                counts['X'] += 1
            # Ignore whitespace, numbers, or other symbols from counting
                
    return counts


def main():
    """
    Main function to handle command-line arguments and orchestrate the counting.
    """
    # 1. Handle command-line arguments (sys)
    if len(sys.argv) < 2:
        print(f"Usage: python {os.path.basename(sys.argv[0])} <input_AA_fasta> [output_file_txt]", file=sys.stderr)
        sys.exit(1)

    input_file = sys.argv[1]
    # Default output file name is used if not provided
    output_file = sys.argv[2] if len(sys.argv) > 2 else "amino_counts.txt"

    print(f"Starting amino acid counting from '{input_file}'...")

    # 2. Read the input FASTA file
    aa_sequences = read_aa_fasta(input_file)
    
    if aa_sequences is None:
        # Errors handled and printed inside read_aa_fasta
        sys.exit(1)
        
    # 3. Count the abundances
    aa_counts = count_abundances(aa_sequences, STANDARD_AAS)
        
    # 4. Write the output to a file (try/except/finally)
    output_success = False
    
    # Separate standard AAs from 'X' for printing order
    standard_counts = {k: v for k, v in aa_counts.items() if k != 'X'}
    x_count = {'X': aa_counts.get('X', 0)}

    try:
        # Using 'with open' for secure file writing
        with open(output_file, 'w') as f:
            
            # Print standard amino acids first (A-Z order because of sorted keys)
            for aa, count in standard_counts.items():
                f.write(f"{aa} {count}\n")
                
            # Print 'X' at the bottom
            f.write(f"X {x_count['X']}\n")
            
            output_success = True
            
    except Exception as e:
        print(f"\nCritical Error: Failed to write to output file '{output_file}'. Error: {e}", file=sys.stderr)
        
    finally:
        if output_success:
            print(f"Successfully wrote amino acid counts to '{output_file}'.")
        else:
            print("Counting process completed with errors during file writing.", file=sys.stderr)

if __name__ == "__main__":
    main()


In [ ]:
import sys
import os
import itertools # for grouping FASTQ lines into 4s

# ----------------------------------------------------------------------
# DOCUMENTATION BLOCK
# Program: barcode.py
# Author: Gemini (as Student II)
# Date: October 2025
# Description: Reads a FASTQ file, trims specified barcodes from the 5' 
# end of sequences, and writes the results to separate FASTQ files for 
# each barcode and an 'undetermined' file for unmatched sequences.
# Requirements: Uses sys, def, if/while, try/except/finally, with open.
# Execution: python barcode.py input.fastq [output_base] [undetermined_file]
# ----------------------------------------------------------------------

# Predefined barcodes (converted to uppercase for case-insensitive matching in practice)
BARCODES = {
    "TATCCTCT": "sample1",
    "GTAAGGAG": "sample2", 
    "TCTCTCCG": "sample3"
}

# Default output file names
DEFAULT_UNDETERMINED_FILE = "undetermined.fastq"
DEFAULT_BASE_NAME = "barcode" # will create e.g., barcode_TATCCTCT.fastq


def fastq_reader(file_path):
    """
    A generator function that yields a FASTQ record (header, seq, plus, quality)
    by reading 4 lines at a time.
    
    Args:
        file_path (str): The path to the input FASTQ file.
        
    Yields:
        tuple: (header, sequence, plus_line, quality_score)
    """
    try:
        # Use 'with open' for robust file reading
        with open(file_path, 'r') as f:
            # Use a while loop to ensure we read in blocks of 4 lines
            while True:
                # Read the four lines of a FASTQ record
                header = f.readline().strip()
                sequence = f.readline().strip()
                plus_line = f.readline().strip()
                quality = f.readline().strip()
                
                # If the header is empty, we've reached EOF
                if not header:
                    break
                    
                # Validate that it looks like a FASTQ record
                if not header.startswith('@') or not plus_line.startswith('+'):
                    # This raises an error which will be caught in the main function
                    raise ValueError("Input file is not in correct FASTQ format (missing @ or + line).")

                # The sequence and quality must be the same length
                if len(sequence) != len(quality):
                    raise ValueError(f"Sequence and quality line length mismatch for record starting with {header}.")

                yield (header, sequence, plus_line, quality)
                
    except FileNotFoundError:
        print(f"Error: Input file not found at '{file_path}'", file=sys.stderr)
        # Re-raise to be handled by the caller/main function
        raise 
    except ValueError:
        raise
    except Exception as e:
        print(f"An unexpected error occurred while parsing the file: {e}", file=sys.stderr)
        raise
        
        
def write_fastq_record(f_handle, header, sequence, plus_line, quality):
    """Writes a single FASTQ record to a file handle."""
    f_handle.write(f"{header}\n")
    f_handle.write(f"{sequence}\n")
    f_handle.write(f"{plus_line}\n")
    f_handle.write(f"{quality}\n")


def main():
    """
    Main function to handle command-line arguments and orchestrate the trimming.
    """
    # 1. Handle command-line arguments (sys)
    if len(sys.argv) < 2:
        print(f"Usage: python {os.path.basename(sys.argv[0])} <input_fastq> [output_base_name] [undetermined_file]", file=sys.stderr)
        sys.exit(1)

    input_file = sys.argv[1]
    # The problem description is slightly confusing on naming; we will use
    # a base name for barcode files and a specific name for the undetermined file.
    output_base = sys.argv[2] if len(sys.argv) > 2 else DEFAULT_BASE_NAME
    undetermined_file = sys.argv[3] if len(sys.argv) > 3 else DEFAULT_UNDETERMINED_FILE
    
    print(f"Starting barcode trimming on '{input_file}'...")
    
    # 2. Prepare output file handles
    file_handles = {}
    
    # Add the undetermined file handle
    file_handles['undetermined'] = undetermined_file
    
    # Add the barcode file handles, naming them clearly
    for barcode in BARCODES.keys():
        # Creates file names like barcode_TATCCTCT.fastq
        file_handles[barcode] = f"{output_base}_{barcode}.fastq"
        
    
    # 3. Process the file and write to outputs (try/except/finally)
    # We will use 'with open' to open all files simultaneously and ensure they close
    # The files list helps manage the handles dynamically
    files_to_open = {name: path for name, path in file_handles.items()}
    file_objects = {}
    total_records = 0
    
    try:
        # Open all output files using nested 'with open' (or the Tip: opening multiple files)
        # Since this is complex with dynamic files, we'll open them sequentially and close in 'finally'
        # The simplest way to satisfy 'with open' and 'try/except' for dynamic files:

        for name, path in files_to_open.items():
            file_objects[name] = open(path, 'w')
        
        # Start processing input FASTQ records
        for header, sequence, plus_line, quality in fastq_reader(input_file):
            total_records += 1
            
            found_barcode = None
            
            # Check for barcode match (use while loop over barcodes)
            barcodes_iter = iter(BARCODES.keys())
            while found_barcode is None:
                try:
                    current_barcode = next(barcodes_iter)
                    
                    # Case-insensitive check (convert sequence to upper once)
                    if sequence.upper().startswith(current_barcode):
                        found_barcode = current_barcode
                        break
                        
                except StopIteration:
                    # All barcodes checked, no match found
                    break

            
            if found_barcode:
                # Trim the sequence and quality
                barcode_len = len(found_barcode)
                trimmed_sequence = sequence[barcode_len:]
                trimmed_quality = quality[barcode_len:]
                
                # Write to the specific barcode file
                output_handle = file_objects[found_barcode]
                write_fastq_record(output_handle, header, trimmed_sequence, plus_line, trimmed_quality)
                
            else:
                # No match found, write original record to undetermined file
                output_handle = file_objects['undetermined']
                write_fastq_record(output_handle, header, sequence, plus_line, quality)
        
    except FileNotFoundError as e:
        print(f"\nCritical Error: Input file error. {e}", file=sys.stderr)
        sys.exit(1)
    except ValueError as e:
        print(f"\nCritical Error: Invalid FASTQ format. {e}", file=sys.stderr)
        sys.exit(1)
    except Exception as e:
        print(f"\nCritical Error during processing: {e}", file=sys.stderr)
        sys.exit(1)
        
    finally:
        # The 'finally' block ensures all dynamically opened files are closed
        for name, f_obj in file_objects.items():
            if f_obj:
                f_obj.close()
                print(f"Closed output file: {file_handles[name]}")
        
        print(f"\nProcessing complete. Total records processed: {total_records}.")

if __name__ == "__main__":
    main()
